In [ ]:
import json
import os
import pandas as pd
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from IPython.display import clear_output

In [ ]:
client_id = "174068e000474d4ca4531e5139e4dfc4"  
client_secret = "fab2340588c143a38f65b027c0a27df6"

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
def get_track_feature(track_id_array):
    return sp.audio_features(track_id_array)

In [ ]:

filter_data = pd.read_csv("../../../handle/song_data.csv")
update_data = filter_data[filter_data['danceability'].isnull()]
link_arr = list(update_data["spotify_link"])
parts = [link_arr[i:i+50] for i in range(0, len(link_arr), 50)]

In [ ]:
fetch_data = []
for id, part in enumerate(parts):
###### 1:
    if id < 1999 or id > 3500:
        continue
########
    try:
        print(id)
        fetch_data.append(get_track_feature(part))
        clear_output(wait=True)
    except Exception as e:
        print(f"An error occurred: {e}")  
        break
fetch_data = [x for xs in fetch_data for x in xs]

In [ ]:
for id, feature_dict in enumerate(fetch_data):
    try:
        print(f'{id}/{len(fetch_data)}')
        track_id = feature_dict['id']
        matching_row = update_data[update_data['spotify_link'] == track_id]
        if not matching_row.empty:
            common_features = set(update_data.columns) & set(feature_dict.keys())
            for feature in common_features:
                if feature != 'id':
                    update_data.loc[matching_row.index, feature] = feature_dict[feature]
    except TypeError as e:
        print(f"Caught a TypeError: {e}") 
        continue
    except Exception as e:
        print(f"An error occurred: {e}")
        continue
    clear_output(wait=True)
update_data.to_csv('../../../handle/song_data3.csv', index=False)